<a href="https://colab.research.google.com/github/antonpolishko/task-ties/blob/master/EDA_NER_sections.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import os
import glob
import re

In [0]:
from google.colab import drive
drive.mount('/content/drive')

covid_main = '/content/drive/My Drive/COVID-19'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
section_text_dir = os.path.join(covid_main,'v6_text')
files = glob.glob(os.path.join(covid_main,'v6_text','*.pkl'))

In [0]:
# frames = []
# for i in range(len(files)):
#   df = pd.read_pickle(files[i], compression="gzip")
#   df = df[['paper_id','section','sentence','DISEASE']]
#   frames.append(df)
#   del df

# section_text = pd.concat(frames) 
# section_text.head(10)
# section_text.reset_index(drop=True, inplace=True)
# section_text.to_json(os.path.join(covid_main,'AllSectionText.json'))

In [0]:
frames = []
for i in range(len(files)):
  df = pd.read_pickle(files[i], compression="gzip")
  df = df[['paper_id','section','sentence']]
  df = df[df['section']=='methods']
  frames.append(df)
  del df

methods_sentences = pd.concat(frames).reset_index(drop=True)
methods_sentences.head(10)

,paper_id,section,sentence
0,dd1217361ab14e65057a7b87d32fd866248186fd,methods,We performed simulations of a dental hygiene p...
1,dd1217361ab14e65057a7b87d32fd866248186fd,methods,Aerosols were formed using the Mini-Clean devi...
2,dd1217361ab14e65057a7b87d32fd866248186fd,methods,We placed the aerosol distributor at a distanc...
3,dd1217361ab14e65057a7b87d32fd866248186fd,methods,We used a mannequin head to simulate the denti...
4,dd1217361ab14e65057a7b87d32fd866248186fd,methods,The mannequin's oral cavity was covered with l...
5,dd1217361ab14e65057a7b87d32fd866248186fd,methods,"A vacuum pump (Cattani SpA, Parma, Italy) atta..."
6,dd1217361ab14e65057a7b87d32fd866248186fd,methods,We adjusted the airflow rates using the lock n...
7,dd1217361ab14e65057a7b87d32fd866248186fd,methods,"Using scanning electron microscopy, we found t..."
8,dd1217361ab14e65057a7b87d32fd866248186fd,methods,"We tion, we activated the particulate for only..."
9,dd1217361ab14e65057a7b87d32fd866248186fd,methods,We carefully fitted a new mask for each separa...


In [0]:
# methods_sentences.to_json(os.path.join(covid_main,'methods_sentences.json'))

In [0]:
methods_sentences = pd.read_json(os.path.join(covid_main,'methods_sentences.json'))

In [0]:
#get number of papers that have a methods section
len(methods_sentences['paper_id'].unique())

9692

In [0]:
def get_age(text):
  match1 = re.search(r'(median age|mean age)(.*?)([0-9]+?\s*?years|[0-9]+?\s*?yrs|[0-9]+?\s*?y|months|weeks|days)', text)
    
  match2 = re.search(r'([0-9]+?\s*?years|[0-9]+?\s*?yrs|[0-9]+?\s*?y|months|weeks|days)(\s)(median age|mean age)', text)

  match3 = re.search(r'(median age|mean age)(.*?)([0-9]+?\s*?)', text)
  if match1:
    return match1.group()
  elif match2:
    return match2.group()
  elif match3:
    return match3.group()
  else:
    pass

def get_sex(text):
  match1 = re.search(r'([1-9]+ males)(.*?)([1-9]*? females)|([1-9]+ men)(.*?)([1-9]*? women)', text)
    
  match2 = re.search(r'(\d+(\.\d+)?% males)(.*?)(\d+(\.\d+)?% females)|(\d+(\.\d+)?% men)(.*?)(\d+(\.\d+)?% women)', text)

  match3 = re.search(r'(\d+(\.\d+)?%)(\s)(males|females)|(\d+(\.\d+)?%)(\s)(men|women)', text)

  if match1:
    return match1.group()
  elif match2:
    return match2.group()
  elif match3:
    return match3.group()
  else:
    pass

<class 'pandas.core.frame.DataFrame'>
Int64Index: 187090 entries, 0 to 187089
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   paper_id    187090 non-null  object
 1   section     187090 non-null  object
 2   sentence    187090 non-null  object
 3   age_exists  306 non-null     object
 4   age         298 non-null     object
dtypes: object(5)
memory usage: 8.6+ MB


In [0]:
methods_sentences['age_exists']=methods_sentences['sentence'].apply(lambda x: x if any(s in x for s in ['mean age', 'median age']) else np.nan)
methods_sentences['age'] = methods_sentences['sentence'].apply(lambda x: get_age(x) if pd.notnull(x) else x)
methods_sentences['sex_exists']=methods_sentences['sentence'].apply(lambda x: x if any(s in x.split() for s in ['males', 'females', 'male', 'female', 'men','women']) else np.nan)
methods_sentences['sex'] = methods_sentences['sentence'].apply(lambda x: get_sex(x) if pd.notnull(x) else x)
methods_sentences.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 187090 entries, 0 to 187089
Data columns (total 7 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   paper_id    187090 non-null  object
 1   section     187090 non-null  object
 2   sentence    187090 non-null  object
 3   age_exists  306 non-null     object
 4   age         298 non-null     object
 5   sex_exists  1446 non-null    object
 6   sex         91 non-null      object
dtypes: object(7)
memory usage: 16.4+ MB


In [0]:
methods_sentences[pd.notnull(methods_sentences['age_exists'])]['age_exists'].tolist()

['A total of 336 specimens were evaluated for the presence of HBoV, including 225 virus-negative specimens (median age of patients = 14 months; range 1-69 months) and 111 virus-positive specimens (median age of patients = 15 months; range 1-83 months).',
 'The mean age (± SD) of the male subjects (n = 59) was 24.0 years (± 2.7 years), and that of the female subjects (n = 141) was 24.1 years (± 3.6 years).',
 'Of all the cases, the mean age AE SD was 33.7 AE 4.3 years and the gestation age was 26.3 AE 9 weeks, and there were four (36%) primi-gravida.',
 'The mean age was 52.50±14.55 (22−68) years.',
 'Between June 2012 and July 2013 eighteen critically ill patients were enrolled with a mean age of 56 years and mean APACHE II score of 15 (Table 1) .',
 'The median age was 8.5 months, and 9/10 (was 8.5 months, and 9/10',
 'To analyse the performance of the BNI and BNR compared to RT-PCR we selected 521 nasal washings of 489 patients with a median age of 4 months (minimum 0.03-maximum 58 m

In [0]:
methods_sentences[pd.notnull(methods_sentences['age'])]['age'].tolist()

['median age of patients = 14 months',
 'mean age (± SD) of the male subjects (n = 59) was 24.0 years',
 'mean age AE SD was 33.7 AE 4.3 years',
 'mean age was 5',
 'mean age of 56 years',
 'median age was 8.5 months',
 'median age of 4 months',
 'mean age of study participants was 33 years',
 'mean age was 68.8 years',
 'median age, 34 years',
 'mean age of 4.4 years',
 'median age of 51.08 years',
 'mean age, 38.2 years',
 'median age was 3',
 'median age was 33 (range: 0-90 years',
 'median age was 24 (range: 0-98 years',
 'median age of H1N1sw positive patients: 21; sex ratio: 1.07) and very few cases (less than 1%) were identified in patients over 60 years',
 'mean age 46.29 +/-11.02y',
 'mean age 30 +/-6.7 y',
 'mean age 42.4 +/-11.8 yrs',
 'median age was 2',
 'median age 8.9 years',
 'mean age, 52 years',
 'mean age, 68 ± 11 years',
 'mean age 8.71 ± 3.26 years',
 'median age was 61 years',
 'mean age of study participants was 33 years',
 'mean age of 38 years',
 'mean age of 6

In [0]:
methods_sentences[pd.notnull(methods_sentences['sex_exists'])]['sex_exists'].tolist()

['As a first step in including spatial uncertainty, we selected a square with 25 mile sides for a representation of a very large home range for a male cougar.',
 'The mean age (± SD) of the male subjects (n = 59) was 24.0 years (± 2.7 years), and that of the female subjects (n = 141) was 24.1 years (± 3.6 years).',
 'Eighty 24-week-old male Sprague-Dawley rats (body weight 500-550 g) were housed in a temperature-controlled room (25°C) under a 12/12 h reversed day/night cycle and received food and water ad libitum.',
 'Ceca are usually present, and elements of the male and female reproductive systems in these typically hermaphroditic adult worms (excepting the schistosomes) may be seen.',
 'Schistosomes are recognized by their intravascular location and sexual dimorphism, the leaf-like male perhaps enveloping the slender cylindrical female within the gynecophoric canal, in section.',
 'The first patient was a 29-year-old female (gravida 2, para 1) at 6 weeks gestation and with no underl

In [0]:
methods_sentences[pd.notnull(methods_sentences['sex'])]['sex'].tolist()

['7 males, 5 females',
 '63 males and 25 females',
 '37 males and 19 females',
 '25 males and 28 females',
 '5 males, M1 to M5, and 5 females',
 '5 males, M1 to M5, and 5 females',
 '5 males, M6 to M10, and 5 females',
 '6 males, M5 and M22 to M26, and 4 females',
 '48 males; 25 females',
 '35 males and 25 females',
 '2 males and 2 females',
 '3 males), or on day 3 postexposure (group 3; n = 6, 3 females',
 '8 males, 11 females',
 '61 males and 39 females',
 '32 males and 18 females',
 '772 males and 18,614 females',
 '16 men and 9 women',
 '67 males and 53 females',
 '78 males, 45 females',
 '59 men (22) and women',
 '13 men and 17 women',
 '40.2% men and 59.8% women',
 '417 males (55.97%), and 328 females',
 '38 males and 55 females',
 '80% women',
 '7 men and 3 women',
 '11 males with 150 females',
 '213 males (66 % of the colony) and 296 females',
 '25 men (69.4%) and 11 women',
 '41% males',
 '15 men and 20 women',
 '45% men',
 '55% men',
 '32 men and 32 women',
 '19 males and 36 

In [0]:
methods_sentences.to_json(os.path.join(covid_main,'methods_sentences.json'))

In [0]:
frames = []
for i in range(len(files)):
  df = pd.read_pickle(files[i], compression="gzip")
  df = df[['paper_id','section','sentence']]
  df = df[df['section']=='results']
  frames.append(df)
  del df

results_sentences = pd.concat(frames).reset_index(drop=True)
results_sentences.head(10)

,paper_id,section,sentence
0,1d7079e54330e89dde86b8677dd3c5f676aa7159,results,This section presents the performance of the o...
1,1d7079e54330e89dde86b8677dd3c5f676aa7159,results,The model is evaluated based on four performan...
2,1d7079e54330e89dde86b8677dd3c5f676aa7159,results,The DR shows the model accurateness to detect ...
3,1d7079e54330e89dde86b8677dd3c5f676aa7159,results,The FAR measures how much false detection occu...
4,1d7079e54330e89dde86b8677dd3c5f676aa7159,results,"Lastly, the ACC"
5,b0f36a47c75838cb7c148f87f86a7a879cdc7665,results,CAP is a life-threatening condition that requi...
6,b0f36a47c75838cb7c148f87f86a7a879cdc7665,results,"Underlying comorbid conditions (most commonly,..."
7,b0f36a47c75838cb7c148f87f86a7a879cdc7665,results,The most common etiologic agents found in CAP ...
8,b0f36a47c75838cb7c148f87f86a7a879cdc7665,results,The early and rapid initiation of empiric anti...
9,b0f36a47c75838cb7c148f87f86a7a879cdc7665,results,Initial antimicrobial therapy should consist o...


In [0]:
#get sentences that mention age-related keywords
results_sentences['age_exists']=results_sentences['sentence'].apply(lambda x: x if any(s in x for s in ['mean age', 'median age']) else np.nan)
results_sentences['age'] = results_sentences['sentence'].apply(lambda x: get_age(x) if pd.notnull(x) else x)
results_sentences['sex_exists']=results_sentences['sentence'].apply(lambda x: x if any(s in x.split() for s in ['males', 'females', 'male', 'female', 'men','women']) else np.nan)
results_sentences['sex'] = results_sentences['sentence'].apply(lambda x: get_sex(x) if pd.notnull(x) else x)
results_sentences.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 178050 entries, 0 to 178049
Data columns (total 7 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   paper_id    178050 non-null  string
 1   section     178050 non-null  object
 2   sentence    178050 non-null  string
 3   age_exists  710 non-null     object
 4   age         694 non-null     object
 5   sex_exists  1627 non-null    object
 6   sex         99 non-null      object
dtypes: object(5), string(2)
memory usage: 9.5+ MB


In [0]:
methods_results_demo = pd.concat([methods_sentences,results_sentences]).reset_index(drop=True)
methods_results_demo = methods_results_demo.drop(['age_exists','sex_exists'], axis = 1)
methods_results_demo = methods_results_demo.dropna(subset=['age','sex'], how='all').reset_index(drop=True)
methods_results_demo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1139 entries, 0 to 1138
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   paper_id  1139 non-null   object
 1   section   1139 non-null   object
 2   sentence  1139 non-null   object
 3   age       992 non-null    object
 4   sex       190 non-null    object
dtypes: object(5)
memory usage: 44.6+ KB


In [0]:
len(methods_results_demo['paper_id'].unique())

789

In [0]:
methods_results_demo.head()

,paper_id,section,sentence,age,sex
0,177dc1a77c417696ac36875b7fc3cefec3ec8d55,methods,A total of 336 specimens were evaluated for th...,median age of patients = 14 months,None
1,4519998f22e00e318090334979b5d837a0e10da4,methods,The mean age (± SD) of the male subjects (n = ...,mean age (± SD) of the male subjects (n = 59) ...,None
2,119ad4a03b29b277378ae65af63b8c06ec0fe47a,methods,"Of all the cases, the mean age AE SD was 33.7 ...",mean age AE SD was 33.7 AE 4.3 years,None
3,22616110b79baaf4c9d23f130a9ee35c64d991f7,methods,The mean age was 52.50±14.55 (22−68) years.,mean age was 5,None
4,3d0023b0e5f3b5a5d544abfede465a54fcf2f0bb,methods,Between June 2012 and July 2013 eighteen criti...,mean age of 56 years,None


In [0]:
methods_results_demo[['age','sex']] = methods_results_demo[['age','sex']].applymap(lambda x: x if pd.notnull(x) else "")
methods_results_demo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1139 entries, 0 to 1138
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   paper_id  1139 non-null   object
 1   section   1139 non-null   object
 2   sentence  1139 non-null   object
 3   age       1139 non-null   object
 4   sex       1139 non-null   object
dtypes: object(5)
memory usage: 44.6+ KB


In [0]:
methods_results_demo_combined = methods_results_demo.groupby(['paper_id']).agg(lambda x: ' '.join(x)).reset_index()
methods_results_demo_combined.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 789 entries, 0 to 788
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   paper_id  789 non-null    object
 1   section   789 non-null    object
 2   sentence  789 non-null    object
 3   age       789 non-null    object
 4   sex       789 non-null    object
dtypes: object(5)
memory usage: 30.9+ KB


In [0]:
# methods_results_demo_combined.to_json(os.path.join(covid_main,'methods_results_demographics.json'))
methods_results_demo_combined.to_csv(os.path.join(covid_main,'methods_results_demographics.csv'), index=False)

In [0]:
methods_results_demo_combined.head()

,paper_id,section,sentence,age,sex
0,007e325b2514afd152c8e396b96df06d05147dfd,results,The 1860s (N95) mask was tested in 82 female n...,,"12 males, and the 8233 (N100) mask in 79 females"
1,009892e02bc1a4c9abf6f547b979e68ecbde8087,methods,(4) The subjects' median age at inclusion was ...,median age at inclusion was 6.4 months,
2,00c19aff3efc6e6430f77d81580ac927990ad5a2,results,One hundred fifty-two patents (70.1%) were mal...,mean age was 51.7 ± 14.6 years,
3,01ad54cabdcd7e03fa2f4bf689137cca0f4a4729,results,The mean age of these patients was 55 AE 17.9 ...,mean age of these patients was 55 AE 17.9 years,
4,01cff21366d475b5bc294c90da89a521b510234a,methods,Both genders were represented in almost equal ...,median age was 6 years,52.3% females


In [0]:
# section_text = pd.read_json(os.path.join(covid_main,'AllSectionText.json'))

In [0]:
section_text.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7488469 entries, 0 to 7488468
Data columns (total 5 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   paper_id     object
 1   sentence_id  object
 2   section      object
 3   sentence     object
 4   DISEASE      object
dtypes: object(5)
memory usage: 342.8+ MB


In [0]:
len(section_text['paper_id'].unique())

36533

In [0]:
section_text['section'].unique().tolist()

['table 6',
 'figref0',
 'figref1',
 'tabref0',
 'tabref1',
 'tabref2',
 'tabref3',
 'tabref4',
 'tabref5',
 'tabref6',
 'title',
 'abstract',
 'introduction',
 'reproductive ratio as a predictor of epidemic potential. indeterminacies in transmission events',
 'rates of virus evolution in nature',
 'influence of the time of sampling',
 'interhost versus intrahost rate of evolution',
 'm o d e l s f o r n o n l i n e a r r a t e s o f e v o l u t i o n',
 'rate discrepancies and the clock hypothesis',
 'long-term antigenic diversification of viruses',
 '.1 widely different number of serotypes among genetically variable viruses',
 'monoclonal antibody-resistant mutant (marm) frequencies references',
 'statistics',
 'phylogenetic relationships among viruses. evolutionary models',
 'extinction, survival, and emergence of viral pathogens. back to the mutant clouds',
 'factors in viral emergence',
 'f a c t o r s i n t h e e m e r g e n c e o f m i c r o b i a l d i s e a s e',
 'complexity 

In [0]:
'results' in section_text['section'].unique().tolist()

True

In [0]:
'table 1' in section_text['section'].unique().tolist()

True

In [0]:
titles = section_text[section_text['section']=='title']
titles['DISEASE'] = titles['DISEASE'].apply(lambda x: eval(str(x)))
titles_joined = titles.groupby('paper_id').agg({'DISEASE':sum}).reset_index()
titles_joined['title_most_freq_dz'] = titles_joined['DISEASE'].apply(lambda lst: max(set(lst), key=lst.count) if len(lst)>0 else np.nan)
titles_joined.drop(['DISEASE'], axis = 1, inplace=True)
print(titles_joined.head())
print()
print(len(titles_joined['paper_id'].unique()))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


                                   paper_id title_most_freq_dz
0  0001418189999fea7f7cbe3e82703d71c85a6fe5                NaN
1  00016663c74157a66b4d509d5c4edffd5391bbe0                NaN
2  0005d253951fedc237715a37db147032eea28912                NaN
3  000affa746a03f1fe4e3b3ef1a62fdfa9b9ac52a         infections
4  000b7d1517ceebb34e1e3e817695b6de03e2fa78                NaN

35979


In [0]:
abstracts = section_text[section_text['section']=='abstract']
abstracts['DISEASE'] = abstracts['DISEASE'].apply(lambda x: eval(str(x)))
abstracts_joined = abstracts.groupby('paper_id').agg({'DISEASE':sum}).reset_index()
abstracts_joined['abs_most_freq_dz'] = abstracts_joined['DISEASE'].apply(lambda lst: max(set(lst), key=lst.count) if len(lst)>0 else np.nan)
abstracts_joined.drop(['DISEASE'], axis = 1, inplace=True)
print(abstracts_joined.head())
print()
print(len(abstracts_joined['paper_id'].unique()))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


                                   paper_id                             abs_most_freq_dz
0  0001418189999fea7f7cbe3e82703d71c85a6fe5                                          FIP
1  00016663c74157a66b4d509d5c4edffd5391bbe0                                          NaN
2  0005d253951fedc237715a37db147032eea28912                                          NaN
3  000affa746a03f1fe4e3b3ef1a62fdfa9b9ac52a  methicillin-resistant Staphylococcus aureus
4  000b7d1517ceebb34e1e3e817695b6de03e2fa78                                          NaN

35980


In [0]:
methods = section_text[section_text['section']=='methods']
# print(methods.info())
methods_joined = methods.groupby('paper_id').agg({'sentence': lambda x: ' '.join(x)}).reset_index().reset_index(drop=True)
methods_joined.columns = ['paper_id','methods_sentence']
print(methods_joined.head())
print()
print(len(methods_joined['paper_id'].unique()))


                                   paper_id                                   methods_sentence
0  00016663c74157a66b4d509d5c4edffd5391bbe0  An important aspect of studying diseases inter...
1  000e754142ba65ef77c6fdffcbcbe824e141ea7b  This study was performed at a 2,700-bed, terti...
2  001259ae6d9bfa9376894f61aa6b6c5f18be2177  Cells and Viruses The origin and growth of the...
3  001ec025dd65db8a47827c8f49d9c41d60f35d00  From each sample, 35 μg of acetone-precipitate...
4  0023258070e16dc764316f6797bfd03fea23ee07  This assessment of the impact of a child's ILI...

9692


In [0]:
subjects = section_text[section_text['section']=='subjects'].reset_index(drop=True)
subjects_joined = subjects.groupby('paper_id').agg({'sentence': lambda x: ' '.join(x)}).reset_index().reset_index(drop=True)
print(subjects_joined.head())
print()
print(len(subjects_joined['paper_id'].unique()))

                                   paper_id                                           sentence
0  00016663c74157a66b4d509d5c4edffd5391bbe0  Even though there is no real treatment for nor...
1  000affa746a03f1fe4e3b3ef1a62fdfa9b9ac52a  Yearly aggregated data on the number of nondup...
2  0014b643b47d9b881e401ad5c4995622e0a39433  exhibit a wide range of clinical manifestation...
3  0023258070e16dc764316f6797bfd03fea23ee07  Of 381 children enrolled, 340 were enrolled th...
4  003fe41364ba16eb711b30e687258b90ab085ea6  Interestingly, prevalence of EBV infection was...

2850


In [0]:
results = section_text[section_text['section']=='results']
# print(methods.info())
results_joined = results.groupby('paper_id').agg({'sentence': lambda x: ' '.join(x)}).reset_index().reset_index(drop=True)
results_joined.columns = ['paper_id','results_sentence']
print(results_joined.head())
print()
print(len(results_joined['paper_id'].unique()))

                                   paper_id                                   results_sentence
0  0001418189999fea7f7cbe3e82703d71c85a6fe5  FIPV positive cells were found in cell suspens...
1  000e754142ba65ef77c6fdffcbcbe824e141ea7b  During the study period, there were 453,075 ad...
2  001259ae6d9bfa9376894f61aa6b6c5f18be2177  The characteristics of the MHV-JHM ts mutants ...
3  0028e606679c9525ccc7d9ecc881f02bf49eb3ab  To add depth to our understanding of STIV repl...
4  002faa16056227423f9d108d01f0dae31491510c  In 2005, the influenza vaccination rate for al...

7820


In [0]:
table1 = section_text[section_text['section']=='table 1']
# print(methods.info())
table1_joined = table1.groupby('paper_id').agg({'sentence': lambda x: ' '.join(x)}).reset_index().reset_index(drop=True)
table1_joined.columns = ['paper_id','table1_sentence']
print(table1_joined.head())
print()
print(len(table1_joined['paper_id'].unique()))

                                   paper_id                                    table1_sentence
0  002160a433024de8c5e80b210f0b54e5d029fa57                   Default parameters in the study.
1  03d37bd0d8f532abc81284a3bdd38afc3e376873  Comparison of the mean viral detection achieve...
2  08272a8e6878f62be9d80da33a9c5f8d3561cccc  Possible causes of febrile syndromes in patien...
3  0a512a4819c586666dfab93b1f5ee4fc02004494                              The specific primers.
4  0aa4edc168fd9bf7ba7efee20dbeb007c26b1beb    The oligonucleotide primers used in this study.

79


In [0]:
all_section_data = titles_joined.merge(abstracts_joined, on='paper_id', how='outer')\
.merge(methods_joined, on='paper_id', how='outer').reset_index(drop=True)\
.merge(results_joined, on='paper_id', how='outer').reset_index(drop=True)\
.merge(table1_joined, on='paper_id', how='outer').reset_index(drop=True)
all_section_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36062 entries, 0 to 36061
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   paper_id            36062 non-null  object
 1   title_most_freq_dz  12278 non-null  object
 2   abs_most_freq_dz    19438 non-null  object
 3   methods_sentence    9692 non-null   string
 4   results_sentence    7820 non-null   string
 5   table1_sentence     79 non-null     string
dtypes: object(3), string(3)
memory usage: 1.7+ MB


In [0]:
all_section_data.head(10)

,paper_id,title_most_freq_dz,abs_most_freq_dz,methods_sentence,results_sentence,table1_sentence
0,0001418189999fea7f7cbe3e82703d71c85a6fe5,NaN,FIP,<NA>,FIPV positive cells were found in cell suspens...,<NA>
1,00016663c74157a66b4d509d5c4edffd5391bbe0,NaN,NaN,An important aspect of studying diseases inter...,<NA>,<NA>
2,0005d253951fedc237715a37db147032eea28912,NaN,NaN,<NA>,<NA>,<NA>
3,000affa746a03f1fe4e3b3ef1a62fdfa9b9ac52a,infections,methicillin-resistant Staphylococcus aureus,<NA>,<NA>,<NA>
4,000b7d1517ceebb34e1e3e817695b6de03e2fa78,NaN,NaN,<NA>,<NA>,<NA>
5,000e754142ba65ef77c6fdffcbcbe824e141ea7b,hospital-acquired respiratory virus infection,viral infection,"This study was performed at a 2,700-bed, terti...","During the study period, there were 453,075 ad...",<NA>
6,000eec3f1e93c3792454ac59415c928ce3a6b4ad,NaN,NaN,<NA>,<NA>,<NA>
7,001259ae6d9bfa9376894f61aa6b6c5f18be2177,hepatitis,NaN,Cells and Viruses The origin and growth of the...,The characteristics of the MHV-JHM ts mutants ...,<NA>
8,00142f93c18b07350be89e96372d240372437ed9,NaN,viral infections,<NA>,<NA>,<NA>
9,0014b643b47d9b881e401ad5c4995622e0a39433,Arrhythmias,HRS,<NA>,<NA>,<NA>


In [0]:
all_section_data.to_json(os.path.join(covid_main,'AllSectionText_Processed.json'))

In [0]:
all_section_data[pd.notnull(all_section_data['table1_sentence'])]

,paper_id,title_most_freq_dz,abs_most_freq_dz,methods_sentence,results_sentence,table1_sentence
17,002160a433024de8c5e80b210f0b54e5d029fa57,infection,infection,<NA>,<NA>,Default parameters in the study.
545,03d37bd0d8f532abc81284a3bdd38afc3e376873,NaN,diarrhea,<NA>,<NA>,Comparison of the mean viral detection achieve...
1141,08272a8e6878f62be9d80da33a9c5f8d3561cccc,NaN,NaN,<NA>,<NA>,Possible causes of febrile syndromes in patien...
1447,0a512a4819c586666dfab93b1f5ee4fc02004494,NaN,infection,<NA>,<NA>,The specific primers.
1500,0aa4edc168fd9bf7ba7efee20dbeb007c26b1beb,NaN,diarrhea,The molecular characters of BNoV-VP2 strains w...,<NA>,The oligonucleotide primers used in this study.
...,...,...,...,...,...,...
34113,f2e653d48d4c3eb2197f86102e2b5f301b26af38,NaN,necrosis,<NA>,<NA>,A list of primers that were used for the ampli...
34397,f500549a408ef72f48bc8b64cd5d34e43b3b2241,respiratory syndrome,respiratory syndrome,<NA>,The coronavirus 3CLpros and their cleavage sit...,The number of cleavage sites in pp1ab of 5 rep...
34831,f7f8c3a6f0bc371e5a2f9a32940b7eab3ca84904,Duck Tembusu virus,ovarian folliculi,<NA>,<NA>,Primers for the mutation and cloning of NDV an...
35653,fda38c9f81ad46a3dac992e5606d3f306a16596f,NaN,NaN,<NA>,<NA>,Summary of method repeatability and intermedia...


In [0]:
## define rule to determine the main disease discussed in paper
def assign_disease(row):
  if pd.notnull(row['title_most_freq_dz']):
    row['disease'] = row['title_most_freq_dz']
  else:
    row['disease'] = row['abs_most_freq_dz']
all_section_data = all_section_data.applmap(lambda row: assign_disease(row), axis = 1)

In [0]:
## core claim extraction 
# -*- coding: utf-8 -*-
# Import the required libraries

import nltk, re, csv, os
from nltk.corpus import stopwords
from collections import Counter
import json
import numpy as np
from difflib import SequenceMatcher

In [0]:
def check_similarity(a, b):
    return SequenceMatcher(None, a, b).ratio()

sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')

# Define core and non-core words

core_words = ["highlight", "constitute", "suggest", "indicate", "demonstrate", "show", "reveal", "provide", "illustrate", "describe", "conclude", "support", "establish", "propose", "advocate", "determine", "confirm", "argue", "imply", "display", "offer", "highlights", "constitutes", "suggests", "indicates", "demonstrates", "shows", "reveals", "provides", "illustrates", "describes", "concludes", "supports", "establishes", "proposes", "advocates", "determines", "confirms", "argues", "implies", "displays", "offers", "underlines", "underline", "underlined", "overall", "sum", "therefore", "thus", "together", "conclusion", "collectively", "altogether", "conclude", "conclusively", "consequently", "study", "results", "findings", "research", "report", "data", "paper", "observations", "experiment", "publication", "analysis"]

non_core_words = ["sought", "addition", "well-replicated", "replicated", "sample", "aimed", "aims", "questionnaire", "survey", "based", "interviews", "cross-sectional", "participants", "descriptive", "CI ", "%", "interview", "participant", "cc by 3.0", "previously"] 

# Function that seperates all the lines and creates a list of the most frequent words in the provided text

def determine_frequent_words(abstract, path):
    # lines = sent_detector.tokenize(abstract)
    # text = open(path)
    # body = "==== Body"
    # refs = "==== Refs"
    # started = False
    # body_text = ''
    # for line in text:
    #     if refs in line:
    #         started = False
    #     if body in line:
    #         started = True
    #     if started:
    #         body_text += line
    # body_text = body_text.decode('utf-8')
    
    base_words = nltk.tokenize.casual.casual_tokenize(abstracte.lower()) #body_text
    words = [word for word in base_words if word not in stopwords.words()]
    counts = Counter(words)

    frequent_words = []
    for word, count in counts.most_common(10):
        frequent_words.append(word)
    return frequent_words

# This function assigns every sentence with a score based on some parameters

def assign_sentences_score(sentence, frequent_words):
    sentences = {}
    for line in lines:
        score = 0
        words = nltk.tokenize.casual.casual_tokenize(line)
        words = [word for word in words if word != '[' or ']']
        searchObj = re.search( r'(overall|in sum|therefore|thus|together|in conclusion|concluding|taken together|collectively|altogether|taken collectively|to conclude|conclusively|all together|all things considered|everything considered|as a result|consequently|conclusion|thus|as expressed)*.*(the|these|this|the present|our)*(study|results|findings|research|report|data|observation|experiment|publication|analysis|data set|we)+.*(highlight|constitute|suggest|indicate|demonstrate|show|reveal|provide|illustrate|describe|conclude|support|establish|propose|advocate|determine|confirm|argue|impl|display|offer|underline|allow|found|find)+', line, re.I)
        if searchObj != None:
            score += 25
        for word in words:
            if word.lower() in core_words and line.count(word) == 1:
                score += 10
            if word.lower() in frequent_words and line.count(word) == 1:
                score += 5
            if word.lower() in non_core_words:
                score -= 5
        if len(line) >= 400:
            score -= 50
        sentences[line] = score
    return sentences

# In some cases, two sentences end up with the same score, in such a case an extra check is performed to make sure only one sentence is chosen in the end

def perform_extra_check(candidate_sentences, frequent_words):
    less_frequent_words = frequent_words[:5]
    sentences = {}
    for sentence in candidate_sentences:
        score = 0
        words = nltk.tokenize.casual.casual_tokenize(sentence)
        for word in words:
            if word in less_frequent_words:
                score += 1
        sentences[sentence] = score
    return sentences

# When all the sentences have an assigned score, this function goes over all the sentences and picks the sentence(s) with the highest score

def go_over_sentences(sentences):
    scores = sentences.values()
    highest_score = max(scores)
    core_sentence_counter = 0
    candidate_sentences = []
    for sentence, score in sentences.iteritems():
        if score == highest_score:
            core_sentence_counter += 1
    for sentence, score in sentences.iteritems():
        if score == highest_score and core_sentence_counter == 1:
            core_sentence = sentence
        if score == highest_score and core_sentence_counter > 1:
            candidate_sentences.append(sentence)
    if candidate_sentences:
        less_frequent_words = frequent_words[:5]
        sentences = perform_extra_check(candidate_sentences, less_frequent_words)
        scores = sentences.values()
        highest_score = max(scores)
        for sentence, score in sentences.iteritems():
            if score == highest_score:
                core_sentence = sentence
    return core_sentence

# Provide the directory here with all the articles that should be processed

directory='C:/Users/..'
csvfile = open('Results/results_abstract.csv', 'wb')
fieldnames = ['File', 'Sentence']
writer = csv.DictWriter(csvfile, fieldnames=fieldnames, delimiter='|')
writer.writeheader()

for file in os.listdir(directory):
    path = directory + file
    
    text = open(path)

    front = "==== Front"
    body = "==== Body"
    graphical_abstract = "Graphical Abstract"
    started = False
    abstract = ''
    
    for line in text:
        if body in line or graphical_abstract in line:
            started = False
        if started:
            abstract += line
        if front in line:
            started = True
    
    abstract = abstract.decode('utf-8')
    lines, frequent_words = determine_lines_and_frequent_words(abstract, path)
    sentences = assign_sentences_score(lines, frequent_words)
    core_sentence = go_over_sentences(sentences)
    core_sentence = core_sentence.replace('\n', ' ').replace(';',',')
    core_sentence = core_sentence.encode('utf-8')
    
    writer.writerow({'File': file, 'Sentence': core_sentence})
    
# DO NOT FORGET TO DECODE EXTRACTED RESULTS BEFORE NEXT STEP!